In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from joblib import dump, load
from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
###======Settings for different test cases======###
SamplingRate = 100 # need to be changed, 25/50/100
Duration = 2 # need to be changed, 2/4/10
###============###
WindowSize = 2 * SamplingRate
original_SamplingRate = 100
rate = original_SamplingRate/SamplingRate

In [25]:
data = np.load("data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train=x_train.transpose((0,2,1))
x_test=x_test.transpose((0,2,1))


x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()

x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)

# 设置batch size训练集尽量小一点，加快收敛，测试集无所谓，不进行梯度计算
# 大的batch size能够防止loss震荡， 但大的batch size使模型的泛化能力下降,进入局部最优值
train_loader = DataLoader(traindata, batch_size=256, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [26]:
class Conv_com(nn.Module):
    def __init__(self, ):
        super(Conv_com, self).__init__()
        self.conv_1 = nn.Conv1d(in_channels=1,  out_channels=32, kernel_size=3, stride=2)
        self.conv_2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2)
        self.conv_3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3)
        self.MaxPool = nn.MaxPool1d(2)
        self.Flatten = torch.nn.Flatten()

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = F.relu(self.conv_2(out))
        out = F.relu(self.conv_3(out))
        out = self.MaxPool(out)
        out = self.Flatten(out)
        return out


class M_CRNN(nn.Module):
    def __init__(self, ):
        super(M_CRNN, self).__init__()
        self.conv_x = Conv_com()
        self.conv_y = Conv_com()
        self.conv_z = Conv_com()
        self.rnn = nn.RNN(input_size=2208, hidden_size=64)
        self.fla = nn.Flatten()
        self.fc1 = nn.Linear(64, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, data):
        x, y, z = data[:, 0, :], data[:, 1, :], data[:, 2, :]
        x = x.view(-1, 1, x.shape[-1])
        y = y.view(-1, 1, y.shape[-1])
        z = z.view(-1, 1, z.shape[-1])
        x_out = self.conv_x(x)
        y_out = self.conv_y(y)
        z_out = self.conv_z(z)
        new_feature = torch.cat([x_out, y_out, z_out], dim=1)
        new_feature = new_feature.view(-1, 1, new_feature.shape[-1])
        out, _ = self.rnn(new_feature)
        out = self.fla(out)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return F.log_softmax(out, dim=1)

In [27]:
%%time
setup_seed(42)
model = M_CRNN().cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 100 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.4569
time 0.6 sec:
Test set: Average loss: 0.3345, Accuracy: 3338/3708 (90.02%)

Train Epoch: 2, Average Loss: 0.3991
time 0.6 sec:
Test set: Average loss: 0.3253, Accuracy: 3338/3708 (90.02%)

Train Epoch: 3, Average Loss: 0.4019
time 0.6 sec:
Test set: Average loss: 0.3301, Accuracy: 3338/3708 (90.02%)

Train Epoch: 4, Average Loss: 0.3962
time 0.6 sec:
Test set: Average loss: 0.3279, Accuracy: 3338/3708 (90.02%)

Train Epoch: 5, Average Loss: 0.3943
time 0.6 sec:
Test set: Average loss: 0.3320, Accuracy: 3338/3708 (90.02%)

Train Epoch: 6, Average Loss: 0.3963
time 0.6 sec:
Test set: Average loss: 0.3340, Accuracy: 3338/3708 (90.02%)

Train Epoch: 7, Average Loss: 0.4092
time 0.6 sec:
Test set: Average loss: 0.3299, Accuracy: 3338/3708 (90.02%)

Train Epoch: 8, Average Loss: 0.4032
time 0.6 sec:
Test set: Average loss: 0.3331, Accuracy: 3338/3708 (90.02%)

Train Epoch: 9, Average Loss: 0.3987
time 0.6 sec:
Test set: Average loss: 0.3351, Accuracy: 333

Train Epoch: 73, Average Loss: 0.0009
time 0.6 sec:
Test set: Average loss: 0.0004, Accuracy: 3708/3708 (100.00%)

Train Epoch: 74, Average Loss: 0.0010
time 0.6 sec:
Test set: Average loss: 0.0004, Accuracy: 3708/3708 (100.00%)

Train Epoch: 75, Average Loss: 0.0011
time 0.6 sec:
Test set: Average loss: 0.0007, Accuracy: 3708/3708 (100.00%)

Train Epoch: 76, Average Loss: 0.0015
time 0.6 sec:
Test set: Average loss: 0.0006, Accuracy: 3708/3708 (100.00%)

Train Epoch: 77, Average Loss: 0.0002
time 0.6 sec:
Test set: Average loss: 0.0004, Accuracy: 3707/3708 (99.97%)

Train Epoch: 78, Average Loss: 0.0002
time 0.6 sec:
Test set: Average loss: 0.0006, Accuracy: 3707/3708 (99.97%)

Train Epoch: 79, Average Loss: 0.0002
time 0.6 sec:
Test set: Average loss: 0.0001, Accuracy: 3708/3708 (100.00%)

Train Epoch: 80, Average Loss: 0.0001
time 0.6 sec:
Test set: Average loss: 0.0001, Accuracy: 3708/3708 (100.00%)

Train Epoch: 81, Average Loss: 0.0002
time 0.6 sec:
Test set: Average loss: 0.0002

In [22]:
model.eval()
y_pred = []
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)

summary(y_pred,y_test)

confusion matrix:
 [[3338    1]
 [   0  369]]
Roc score: 0.9999
F1 score: 0.9986
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3339
           1       1.00      1.00      1.00       369

    accuracy                           1.00      3708
   macro avg       1.00      1.00      1.00      3708
weighted avg       1.00      1.00      1.00      3708



In [24]:
torch.save(model, './M_CRNN.pth')